---
title: "AWS - Tips and Tricks"
author: "Vahram Poghosyan"
date: "2023-01-13"
categories: ["AWS", "Cloud Infrastructure", "Docker"]
format:
  html:
    code-fold: true
jupyter: python3
include-after-body:
  text: |
    <script type="application/javascript" src="../../javascript/light-dark.js"></script>
---

# The AWS Metadata Service (IMDS)

This service tells us important information about an AWS service in use. For instance, it can be used to retrieve the IP address and instance ID of a running EC2 instance.

Here's an example. 

We can define a shell script `get-env.sh` and include it in the Dockerfile of our service. 

```docker
RUN ["chmod", "u+x,g+x", "/app/bin/set-env.sh"]
```
We can set the entrypoint to another script called `start-service.sh` and *source* `get-env` inside of it. 

```docker
WORKDIR /app
USER appuser
ENTRYPOINT ["/app/entrypoint", "/app/bin/start-service.sh"]
```

Sourcing a script with the dot command (`.`) means that the script is executed in the current shell environment (rather than a sub-shell). This allows any environment variables or functions defined in the sourced script to be available in the current script.

Inside `start-service.sh`:

```bash
. ./bin/set-env.sh
```

Finally, inside the `set-env.sh` we need to call the AWS Metadata Service like so: 

```bash
IMDSV2_TOKEN=$(curl -X PUT "http://169.254.169.254/latest/api/token/" -H "X-aws-ec2-metadata-token-ttl-seconds: 21600")
```

Because the IP address and instance ID are available from within our running instance, there's no need to use the AWS CLI or the AWS Console (UI). Inside the same script, `set-env.sh`, we export the IP address and the instance ID like:

```bash
export EC2_IN_ADDRRESS=$(curl -H "X-aws-ec2-metadata-token: $IMDSV2_TOKEN -v --connect-timeout 3 --max-time 5 http://169.254.169.254/latest/meta-data/local-ipv4")
export EC2_INSTANCE_ID=$(curl -H "X-aws-ec2-metadata-token: $IMDSV2_TOKEN -v --connect-timeout 3 --max-time 5 http://169.254.169.254/latest/meta-data/instance-id")
```


As the service is coming up, using the Dockerfile, it will execute the `set-env.sh` script on the EC2 instance and retrieve its details.